In [1]:
# import dependencies
import datetime as dt
import pandas as pd
import numpy as np
import requests, io, zipfile, os, time
import dask.dataframe as dd
# import urllib.request
# from urllib.error import URLError


In [2]:
# format of url = "https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip"
url_base = 'https://s3.amazonaws.com/tripdata/'

root = 'Resources'

result_files = []
rename_files = []

In [3]:
# files set up by month, get by 4 digit year and 2 digit month
d = dt.date.today()
url_year = d.year
url_month = d.month - 1  #current month not in file
url_file_prefix = f'{url_year:04d}{url_month:02d}'
prefix = url_file_prefix

# function for December file to create previous year January file info
# def get_next_prefix(prefix):

    
# of months to loop through
for i in range (24):
    prefix == url_file_prefix
    if i != 0:
        # loop back for the next month's file
        url_year = int(prefix[:4])
        url_month = int(prefix[-2:])
            # on December, next previous is January of previous year
        if url_month == 12:
            url_year -= 1
            url_month -= 11
        else:
            url_month -= 1
        url_file_prefix = f'{url_year:04d}{url_month:02d}'
    
    # set up file name
    new_file = url_file_prefix + '.csv'

    # directory files same through 2016, changed for 2017
    url_file_2016 = '-citibike-tripdata.zip'
    url_file_2017 = '-citibike-tripdata.csv.zip'

    # build the proper url to get data from files in the directory
    if url_year >= 2017:
        url = url_base + url_file_prefix + url_file_2017
    else:
        url = url_base + url_file_prefix + url_file_2016
    print(url) 

    # make request for csv file
    r = requests.get(url)
    un_zipped = zipfile.ZipFile(io.BytesIO(r.content))
    
    # download only the existing file in the root directory
    with un_zipped as zf:
        for file in zf.namelist():
            
            # not in root directory, pass, otherwise save
            if file.startswith('_'):
                pass
            else:
                zf.extract(file, root)                        
                result_files.append(file)
                rename_files.append(new_file)


https://s3.amazonaws.com/tripdata/202303-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/202302-citibike-tripdata.csv.zip


In [4]:
# Function to rename files for clarity and easier looping to combine the data
##  https://www.tutorialspoint.com/rename-multiple-files-using-python

def main():
    for i, file in enumerate(os.listdir(root)):
        source = f'{root}/{result_files[i]}'
        destination = f'{root}/{rename_files[i]}'
        os.rename(source, destination)
# Driver Code
if __name__ == '__main__':     
    # Calling main() function
    main()

In [ ]:
# run the dask dataframes to organize the data
##  dask documentation/code from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
##  NaN values, https://medium.com/@nick_frush/getting-dirty-with-dask-part-1-4ecd3373c29b

# dtypes = {
#         'ride_id': 'object',
#         'rideable_type': 'object',
#         'started_at': 'object',
#         'ended_at': 'object',
#         'start_station_name': 'object',
#         'start_station_id': 'object',
#         'end_station_name': 'object',
#         'end_station_id': 'object',
#         'start_lat': 'object',
#         'start_lng': 'object',
#         'end_lat': 'object',
#         'end_lng': 'object',
#         'member_casual': 'object',                              
#           }

# df_list = []
# for i, csv_file in enumerate(rename_files):
    
#     # remove .csv in df name
#     df_name = (f'dask_{rename_files[i]}_df')
#     df_name = df_name.replace('.csv', '')
#     print(f'{df_name} line 9')
#     df_list.append(df_name)
#     csv_name = rename_files[i]

#     # for curiosity, time taken to read data
#     s_time_dask = time.time()
#     df_name = dd.read_csv(f'{root}/{csv_name}',
#                         delimiter=',',
#                         engine='python',
#                         dtype=dtypes
#                          )
    
#     print(f'{df_name} line 21')
#     e_time_dask = time.time()

#     print(f'{df_name} line 30')
#     print(csv_name, " read with dask: ", (e_time_dask-s_time_dask), "seconds   created DataFrame: ", df_name )
    
#     df_name = df_name.compute()


In [6]:
# create list for csv reads
path_list = []
for i, csv_file in enumerate(rename_files):
    path = f'{root}/{csv_file}'
    path_list.append(path)

# for curiosity, time taken to read data
## dask documentation from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
s_time = time.time()

# Read CSV files from List
df = pd.concat(map(pd.read_csv, path_list))

e_time = time.time()
print("Processed in ", (e_time-s_time), "seconds")

C:\Users\slaroue\anaconda3\envs\PythonData\lib\site-packages\pandas\core\reshape\concat.py:304: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  sort=sort,
C:\Users\slaroue\anaconda3\envs\PythonData\lib\site-packages\pandas\core\reshape\concat.py:304: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  sort=sort,


Processed in  20.842799425125122 seconds


In [7]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EAB9FFEAB7BDBAF2,classic_bike,2023-03-15 18:11:59,2023-03-15 18:18:38,6 Ave & W 33 St,6364.07,9 Ave & W 22 St,6266.06,40.748423,-73.988208,40.745497,-74.001971,member
1,B57211E2408E604F,classic_bike,2023-03-18 21:05:59,2023-03-18 21:10:51,6 Ave & W 33 St,6364.07,W 35 St & 9 Ave,6569.09,40.749134,-73.988536,40.754145,-73.996089,member
2,9411DD44FB7406A9,classic_bike,2023-03-16 10:16:15,2023-03-16 10:24:27,6 Ave & W 33 St,6364.07,W 35 St & 9 Ave,6569.09,40.749013,-73.988484,40.754145,-73.996089,member
3,DC2D4F90D746F435,classic_bike,2023-03-01 06:55:20,2023-03-01 07:05:44,E 5 St & Ave A,5626.06,8 Ave & W 31 St,6450.05,40.724775,-73.983742,40.750585,-73.994685,member
4,63874304EFBD61EE,classic_bike,2023-03-02 23:21:02,2023-03-02 23:42:19,3 Ave & Schermerhorn St,4437.01,Greenpoint Ave & Manhattan Ave,5785.05,40.686803,-73.979754,40.730260,-73.953940,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752143,389430A56EA04541,electric_bike,2023-02-21 12:40:49,2023-02-21 12:43:41,W 22 St & 10 Ave,6306.06,W 22 St & 8 Ave,6224.03,40.746920,-74.004519,40.744751,-73.999154,member
1752144,653158DC3395255E,classic_bike,2023-02-14 16:20:36,2023-02-14 17:03:37,W 47 St & 6 Ave,6667.04,W 51 St & 6 Ave,6740.1,40.758397,-73.982550,40.760660,-73.980420,member
1752145,C3707AC98071DDAD,classic_bike,2023-02-11 16:52:16,2023-02-11 17:12:15,W 22 St & 10 Ave,6306.06,W 22 St & 8 Ave,6224.03,40.746920,-74.004519,40.744751,-73.999154,member
1752146,F3A6D114409FA6A1,classic_bike,2023-02-22 19:25:20,2023-02-22 19:28:54,W 22 St & 10 Ave,6306.06,W 22 St & 8 Ave,6224.03,40.746920,-74.004519,40.744751,-73.999154,member
